# Two Factor Hull White Model Calibration
In this section, we will provide a numerical example of calibration of 2-factor Hull-White model. We will fit the model to historical data of swaptions volatility and obtain the five parameters $\alpha, \beta, \sigma_{1}, \sigma_{2}, \rho$

In [1]:
import QuantLib as ql
from collections import namedtuple
import math

In [2]:
today = ql.Date(15, ql.February, 2020);
settlement= ql.Date(19, ql.February, 2020);
ql.Settings.instance().evaluationDate = today;
term_structure = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement,0.04875825,ql.Actual365Fixed())
    )
index = ql.Euribor1Y(term_structure)

We are going to calibrate to the swaption volatilities as shown below. 

In [3]:
CalibrationData = namedtuple("CalibrationData", 
                             "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
        CalibrationData(2, 4, 0.1108),
        CalibrationData(3, 3, 0.1070),
        CalibrationData(4, 2, 0.1021),
        CalibrationData(5, 1, 0.1000 )]

In [4]:
# <!-- collapse=True -->
def create_swaption_helpers(data, index, term_structure, engine):
    swaptions = []
    fixed_leg_tenor = ql.Period(1, ql.Years)
    fixed_leg_daycounter = ql.Actual360()
    floating_leg_daycounter = ql.Actual360()
    for d in data:
        vol_handle = ql.QuoteHandle(ql.SimpleQuote(d.volatility))
        helper = ql.SwaptionHelper(ql.Period(d.start, ql.Years),
                                   ql.Period(d.length, ql.Years),
                                   vol_handle,
                                   index,
                                   fixed_leg_tenor,
                                   fixed_leg_daycounter,
                                   floating_leg_daycounter,
                                   term_structure
                                   )
        helper.setPricingEngine(engine)
        swaptions.append(helper)
    return swaptions    

def calibration_report(swaptions, data):
    print("-"*82)
    print("%15s %15s %15s %15s %15s" % \
    ("Model Price", "Market Price", "Implied Vol", "Market Vol", "Rel Error"))
    print("-"*82)
    cum_err = 0.0
    for i, s in enumerate(swaptions):
        model_price = s.modelValue()
        market_vol = data[i].volatility
        black_price = s.blackPrice(market_vol)
        rel_error = model_price/black_price - 1.0
        implied_vol = s.impliedVolatility(model_price,
                                          1e-5, 50, 0.0, 0.50)
        rel_error2 = implied_vol/market_vol-1.0
        cum_err += rel_error2*rel_error2
        
        print("%15.5f %15.5f %15.5f %15.5f %15.5f" % \
        (model_price, black_price, implied_vol, market_vol, rel_error))
    print("-"*82)
    print("Cumulative Error : %15.5f" % math.sqrt(cum_err))

## G2++ Model

By \cite{BA1} we know that G2++ model is equivalent to Two-Factor Hull-White model. Since Quantlib only support G2++ at the moment, we will calibrate G2++ model first to get its 5 parameters. After that we can map them uniquely to the set of Hull-White parameters.

G2++ model is given by:
\begin{equation}
dr_t = \varphi(t) + x_t + y_t
\end{equation}
    
where $ x_t $ and $ y_t $ are defined by
 
\begin{align*}
&dx_t = -a x_t dt + \sigma dW^1_t\nonumber \\
&dy_t = -b y_t dt + \eta dW^2_t \nonumber \\
&\left<dW^1_t dW^2_t\right> = \rho dt 
\end{align*}

We use the `TreeSwaptionEngine` to value the swaptions in the calibration step. 

In [5]:
model = ql.G2(term_structure);
engine = ql.TreeSwaptionEngine(model, 25)
swaptions = create_swaption_helpers(data, index, term_structure, engine)

In [6]:
optimization_method = ql.LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
end_criteria = ql.EndCriteria(1000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)

a, sigma, b, eta, rho = model.params()
print("G2++ parameters after calibration are:\na = %6.5f, sigma = %6.5f, b = %6.5f, eta = %6.5f, rho = %6.5f " % (a, sigma, b, eta, rho))

G2++ parameters after calibration are:
a = 0.04810, sigma = 0.00301, b = 0.03892, eta = 0.00472, rho = 0.03977 


## Mapping from G2++ to 2-factor Hull-White parameters
By \cite{BA1} the mapping between G2++ model and 2-factor Hull-White model are as follow:
\begin{align*}
\alpha &= a \\
\beta &= b \\
\sigma_{1} &= \sqrt{\sigma^{2} + \eta^{2} + 2\rho\sigma\eta} \\
\sigma_{2} &= \eta(a - b) \\
\bar{\rho} &= \frac{\sigma\rho + \eta}{\sigma_{1}} \\
\end{align*}

In [7]:
def to_2FWH_params(a, sigma, b, eta, rho):
    alpha = a
    beta = b
    sigma_1 = math.sqrt(sigma**2 + eta**2 + 2*rho*sigma*eta)
    sigma_2 = eta*(a - b)
    rho_bar = (sigma*rho + eta)/sigma_1
    return alpha, beta, sigma_1, sigma_2, rho_bar
test = to_2FWH_params(0.521159, 0.005779, 0.075631, 0.011573, -0.986876)
assert(test == (0.521159, 0.075631, 0.005943560538348711, 0.005156095544000001, 0.9875971748124582))

In [8]:
alpha, beta, sigma_1, sigma_2, rho_bar = to_2FWH_params(a, sigma, b, eta, rho)
print("Two-factor Hull-White parameters after calibration are:\nalpha = %6.5f, beta = %6.5f, sigma_1 = %6.5f, sigma_2 = %6.5f, rho_bar = %6.5f " % (alpha, beta, sigma_1, sigma_2, rho_bar))

Two-factor Hull-White parameters after calibration are:
alpha = 0.04810, beta = 0.03892, sigma_1 = 0.00570, sigma_2 = 0.00004, rho_bar = 0.84939 


In [9]:
calibration_report(swaptions, data)

----------------------------------------------------------------------------------
    Model Price    Market Price     Implied Vol      Market Vol       Rel Error
----------------------------------------------------------------------------------
        0.00871         0.00949         0.10531         0.11480        -0.08263
        0.00968         0.01008         0.10634         0.11080        -0.04018
        0.00867         0.00871         0.10652         0.10700        -0.00448
        0.00653         0.00625         0.10665         0.10210         0.04442
        0.00357         0.00334         0.10680         0.10000         0.06773
----------------------------------------------------------------------------------
Cumulative Error :         0.12288
